In [ ]:
import os, sys, cv2, random
from tqdm.notebook import tqdm
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import check_random_state
from sklearn.model_selection import StratifiedShuffleSplit


from tqdm.auto import tqdm
tqdm.pandas()
seed=2001

In [ ]:
df= pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
df['id_freq'] = df['individual_id'].map(df['individual_id'].value_counts())
print(len(df.species.unique()), len(df.individual_id.unique()))
df.head()

In [ ]:
df.info()

In [ ]:
df.species= df.species.apply(lambda x: 'killer_whale' if x=='kiler_whale' else x)
df.species= df.species.apply(lambda x: 'bottlenose_dolphin' if x=='bottlenose_dolpin' else x)
df.info()

In [ ]:
var= df.species.value_counts()
x,y= var.index, var.values
plt.figure(figsize=(15,8))
plt.title("Species Distribution", size=16)
sns.barplot(y, x, palette='Blues_d')
plt.show()

In [ ]:
#target
sns.histplot(df.id_freq, bins=50)
plt.show()

In [ ]:
i=1
i, (len(df[df.id_freq<=i])/51033)*100, (len(df[df.id_freq>i])/51033)*100

In [ ]:
i=150
i, (len(df[df.id_freq<=i])/51033)*100, (len(df[df.id_freq>i])/51033)*100

In [ ]:
df= df[df.id_freq<=150]

In [ ]:
df.head()

### Triplate Loss

In [ ]:
df.shape

In [ ]:
def generate(df):
    random.seed(seed)
    df_grp= dict(list(df.groupby('individual_id')))
    
    def aux(row):
        single=False
        anchor= row.image
        ids= df_grp[row.individual_id]['image'].tolist()
        if not len(ids)-1:
            single=True
            positive= ids[0]
        else:
            ids.remove(anchor)
            positive= random.choice(ids)
            
        if not single:
            otherId= list(df_grp.keys())
            otherId.remove(row.individual_id)
            neg_group = random.choice(otherId)
            negative = random.choice(df_grp[neg_group].image.tolist())

            return anchor, positive, negative, None, None
        else:
            otherId= list(df_grp.keys())
            otherId.remove(row.individual_id)
            #3x
            k=3
            negative=[]
            for _ in range(k):
                neg_group = random.choice(otherId)
                negative.append(random.choice(df_grp[neg_group].image.tolist()))
            single=False
            return anchor, positive, negative[0], negative[1], negative[2]
    
    return aux

In [ ]:
train_triplets = df.progress_apply(generate(df), axis=1).tolist()
train_triplets_df = pd.DataFrame(train_triplets, columns=['anchor', 'positive', 'negative0', 'negative1', 'negative2'])
train_triplets_df.head()

In [ ]:
train_triplets_df.to_csv('tripletData.csv', index=False)

In [ ]:
df.to_csv('data.csv', index=False)